In this project, we'll work with a **data set of submissions to popular technology site Hacker News.**
We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question.
We'll compare these two types of posts to determine the following:

--- *Do Ask HN or Show HN receive more comments on average?*

--- *Do posts created at a certain time receive more comments on average?*

In [10]:
from csv import reader
opened_file = open('hacker_news.csv', encoding='utf-8')
read_file = reader(opened_file)
hn = list(read_file)

In [11]:
headers = hn[:1]
hn = hn[1:]

In [12]:
print(headers)

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']]


In [13]:
print(hn[:5])

[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16'], ['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']]


In [14]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn') == True:
        ask_posts.append(row)
    elif title.startswith('show hn') == True:
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

9139
10158
273822


Next, let's determine if ask posts or show posts receive more comments on average.

In [15]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments = round(total_ask_comments / len(ask_posts), 2)
print(avg_ask_comments)

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = round(total_show_comments / len(show_posts), 2)
print(avg_show_comments)

10.39
4.89


**On average, ask posts receive more comments than show posts**

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

In [16]:
import datetime as dt

result_list = []

for row in ask_posts:
    new_list = []
    new_list.append(row[6])
    new_list.append(int(row[4]))
    result_list.append(new_list)

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    dt_object = dt.datetime.strptime(row[0], '%m/%d/%Y %H:%M')
    dt_hour = dt_object.strftime('%H')
    if dt_hour in counts_by_hour:
        counts_by_hour[dt_hour] += 1
        comments_by_hour[dt_hour] += row[1]
    else:
        counts_by_hour[dt_hour] = 1
        comments_by_hour[dt_hour] = row[1]
        
print(counts_by_hour)
print(comments_by_hour)

{'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}
{'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '23': 2297, '20': 4462, '16': 4466, '08': 2362, '00': 2277, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [17]:
avg_by_hour = []

for hour in counts_by_hour:
    avg_by_hour.append([hour, round(comments_by_hour[hour] / counts_by_hour[hour], 1)])
    
avg_by_hour

[['02', 11.1],
 ['01', 7.4],
 ['22', 8.8],
 ['21', 8.7],
 ['19', 7.2],
 ['17', 9.4],
 ['15', 28.7],
 ['14', 9.7],
 ['13', 16.3],
 ['11', 9.0],
 ['10', 10.7],
 ['09', 6.7],
 ['07', 7.0],
 ['03', 7.9],
 ['23', 6.7],
 ['20', 8.7],
 ['16', 7.7],
 ['08', 9.2],
 ['00', 7.6],
 ['18', 7.9],
 ['12', 12.4],
 ['04', 9.7],
 ['06', 6.8],
 ['05', 8.8]]

Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [18]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print('Top 5 Hours for Ask Posts Comments')
print('')
for row in sorted_swap[:5]:
    dt_object = dt.datetime.strptime(row[1], '%H')
    dt_string = dt_object.strftime('%H:%M')
    number = row[0]
    print('{}: {} average comments per post'.format(dt_string, number))

Top 5 Hours for Ask Posts Comments

15:00: 28.7 average comments per post
13:00: 16.3 average comments per post
12:00: 12.4 average comments per post
02:00: 11.1 average comments per post
10:00: 10.7 average comments per post
